<a href="https://colab.research.google.com/github/TrentynHowes/Game-Recommendation-/blob/main/FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE

import warnings
warnings.filterwarnings('ignore')

In [2]:
games = pd.read_csv('https://raw.githubusercontent.com/TrentynHowes/Game-Recommendation-/main/Video_Game.csv')
games.head()

,id,Title,Release_Date,Release_Year,Developer,Publisher,Genres,Product_Rating,User_Score,Metric_Rating,Interested
0,1,Ziggurat (2012),2/17/2012,2012,Action Button Entertainment,Freshuu Inc.,Action,RP,69,83,0
1,2,4X4 EVO 2,11/15/2001,2001,Terminal Reality,Gathering,Auto Racing Sim,E,59,59,0
2,3,MotoGP 2 (2001),1/22/2002,2002,Namco,Namco,Auto Racing Sim,E,58,78,0
3,4,Gothic 3,11/14/2006,2006,Piranha Bytes,Aspyr,Western RPG,T,75,63,0
4,5,Siege Survival: Gloria Victis,5/18/2021,2021,FishTankStudio,Black Eye Games,RPG,RP,65,69,0


In [3]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13751 entries, 0 to 13750
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              13751 non-null  int64 
 1   Title           13751 non-null  object
 2   Release_Date    13751 non-null  object
 3   Release_Year    13751 non-null  int64 
 4   Developer       13751 non-null  object
 5   Publisher       13751 non-null  object
 6   Genres          13751 non-null  object
 7   Product_Rating  13751 non-null  object
 8   User_Score      13751 non-null  int64 
 9   Metric_Rating   13751 non-null  int64 
 10  Interested      13751 non-null  int64 
dtypes: int64(5), object(6)
memory usage: 1.2+ MB


In [26]:
Favorite_Games = ["Ziggurat (2012)", "4X4 EVO 2"] #Enter favorite games here
Least_Favorite = ["Fallout 4"] #Enter Least Favorite games here

for F in Favorite_Games:
  games.loc[games['Title'] == F, 'Interested'] = 1

for L in Least_Favorite:
  games.loc[games['Title'] == L, 'Interested'] = -1


In [5]:
games.head()

,id,Title,Release_Date,Release_Year,Developer,Publisher,Genres,Product_Rating,User_Score,Metric_Rating,Interested
0,1,Ziggurat (2012),2/17/2012,2012,Action Button Entertainment,Freshuu Inc.,Action,RP,69,83,1
1,2,4X4 EVO 2,11/15/2001,2001,Terminal Reality,Gathering,Auto Racing Sim,E,59,59,1
2,3,MotoGP 2 (2001),1/22/2002,2002,Namco,Namco,Auto Racing Sim,E,58,78,0
3,4,Gothic 3,11/14/2006,2006,Piranha Bytes,Aspyr,Western RPG,T,75,63,0
4,5,Siege Survival: Gloria Victis,5/18/2021,2021,FishTankStudio,Black Eye Games,RPG,RP,65,69,0


In [6]:
# Reccomendation System Based on Genres
%%capture
games_vectorizer = CountVectorizer()
games_vectorizer.fit(games['Genres'])

In [7]:
def get_similarities(games_name, data):

  # Getting vector for the input game.
  text_array1 = games_vectorizer.transform(data[data['Title']==games_name]['Genres']).toarray()
  num_array1 = data[data['Title']==games_name].select_dtypes(include=np.number).to_numpy()

  # We will store similarity for each row of the dataset.
  sim = []
  for idx, row in data.iterrows():
    name = row['Title']
    interested = row['Interested']

    if (interested == -1):
      sim.append(None)
    else:
      # Getting vector for current game.
      text_array2 = games_vectorizer.transform(data[data['Title']==name]['Genres']).toarray()
      num_array2 = data[data['Title']==name].select_dtypes(include=np.number).to_numpy()

      # Calculating similarities for text as well as numeric features
      text_sim = cosine_similarity(text_array1, text_array2)[0][0]
      num_sim = cosine_similarity(num_array1, num_array2)[0][0]
      sim.append(text_sim + num_sim)

  return sim

In [8]:
def recommend_games(games_name, data=games):
  # Base case
  if games[games['Title'] == games_name].shape[0] == 0:
    print('This game is either not so popular or you\
    have entered invalid_name.\n Some games you may like:\n')

    for game in data.sample(n=5)['Title'].values:
      if(game["Interested"] != -1):
        print(game)
    return


  data['similarity_factor'] = get_similarities(games_name, data)

  data.sort_values(by=['similarity_factor', 'Developer'],
                   ascending = [False, False],
                   inplace=True)

  # First game will be the input game itself as the similarity will be highest.
  display(data[['Title', 'Developer', 'Genres', 'Interested', 'similarity_factor']][2:10])

In [9]:
recommend_games('The Elder Scrolls V: Skyrim')

,Title,Developer,Genres,Interested,similarity_factor
1089,Avernum 2: Crystal Souls,Spiderweb Software,Western RPG,0,1.999894
1073,Operation Abyss: New Tokyo Legacy,Experience Inc.,Western RPG,0,1.999882
1026,Starfield,Bethesda Game Studios,Western RPG,0,1.999850
1110,Neverwinter Nights 2,Obsidian Entertainment,Western RPG,0,1.999742
933,Shin Megami Tensei: Strange Journey,Atlus,Western RPG,0,1.998646
1205,Sovereign Syndicate,Crimson Herring Studios,Western RPG,0,1.998565
1308,Baldur's Gate II: Throne of Bhaal,BioWare,Western RPG,0,1.995527
815,Baldur's Gate,BioWare,Western RPG,-1,1.995313


from matplotlib import pyplot as plt
_df_0['Interested'].plot(kind='hist', bins=20, title='Interested')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['similarity_factor'].plot(kind='hist', bins=20, title='similarity_factor')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('Title').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_3.groupby('Developer').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['Developer'].value_counts()
    for x_label, grp in _df_7.groupby('Title')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('Title')
_ = plt.ylabel('Developer')

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_8['Title'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_8, x='Interested', y='Title', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_9['Developer'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_9, x='Interested', y='Developer', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_10['Title'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_10, x='similarity_factor', y='Title', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_11['Developer'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_11, x='similarity_factor', y='Developer', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [27]:
# Reccomendation System Based on Developer
%%capture
games_vectorizer = CountVectorizer()
games_vectorizer.fit(games['Developer'])

In [28]:
def get_similarities(games_name, data):

  # Getting vector for the input game.
  text_array1 = games_vectorizer.transform(data[data['Title']==games_name]['Developer']).toarray()
  num_array1 = data[data['Title']==games_name].select_dtypes(include=np.number).to_numpy()

  # We will store similarity for each row of the dataset.
  sim = []
  for idx, row in data.iterrows():
    name = row['Title']
    interested = row['Interested']

    if (interested == -1):
      sim.append(None)
    else:
      # Getting vector for current game.
      text_array2 = games_vectorizer.transform(data[data['Title']==name]['Developer']).toarray()
      num_array2 = data[data['Title']==name].select_dtypes(include=np.number).to_numpy()

      # Calculating similarities for text as well as numeric features
      text_sim = cosine_similarity(text_array1, text_array2)[0][0]
      num_sim = cosine_similarity(num_array1, num_array2)[0][0]
      sim.append(text_sim + num_sim)

  return sim

In [29]:
def recommend_games(games_name, data=games):
  # Base case
  if games[games['Title'] == games_name].shape[0] == 0:
    print('This game is either not so popular or you\
    have entered invalid_name.\n Some games you may like:\n')

    for game in data.sample(n=5)['Title'].values:
      print(game)
    return

  data['similarity_factor'] = get_similarities(games_name, data)

  data.sort_values(by=['similarity_factor', 'Developer'],
                   ascending = [False, False],
                   inplace=True)

  # First game will be the input game itself as the similarity will be highest.
  display(data[['Title', 'Release_Date', 'Developer', 'Genres', 'Product_Rating', 'User_Score', 'Metric_Rating', 'similarity_factor']][2:10])

In [30]:
recommend_games('The Elder Scrolls V: Skyrim')

,Title,Release_Date,Developer,Genres,Product_Rating,User_Score,Metric_Rating,similarity_factor
799,The Elder Scrolls V: Skyrim - Hearthfire,9/4/2012,Bethesda Game Studios,Western RPG,M,58,69,1.994260
1545,Fallout 3: Point Lookout,6/23/2009,Bethesda Game Studios,Western RPG,M,79,79,1.985338
544,The Elder Scrolls V: Skyrim - Dragonborn,12/4/2012,Bethesda Game Studios,Western RPG,M,82,82,1.975930
2686,Fallout 4 VR,12/11/2017,Bethesda Game Studios,Western RPG,RP,50,78,1.903571
3921,Fallout 4: Far Harbor,5/19/2016,Bethesda Game Studios,Western RPG,M,66,78,1.818687
4337,Fallout 3 - Operation: Anchorage,1/27/2009,Bethesda Game Studios,Western RPG,M,66,65,1.794309
5489,The Elder Scrolls: Blades,5/14/2020,Bethesda Game Studios,Western RPG,T,26,26,1.741968
5948,Fallout 76,11/14/2018,Bethesda Game Studios,Western RPG,M,28,52,1.724807


In [31]:
# Reccomendation System Based on Product Rating
%%capture
games_vectorizer = CountVectorizer()
games_vectorizer.fit(games['Product_Rating'])

In [32]:
def get_similarities(games_name, data):

  # Getting vector for the input game.
  text_array1 = games_vectorizer.transform(data[data['Title']==games_name]['Product_Rating']).toarray()
  num_array1 = data[data['Title']==games_name].select_dtypes(include=np.number).to_numpy()

  # We will store similarity for each row of the dataset.
  sim = []
  for idx, row in data.iterrows():
    name = row['Title']
    interested = row['Interested']

    if (interested == -1):
      sim.append(None)
    else:
      # Getting vector for current game.
      text_array2 = games_vectorizer.transform(data[data['Title']==name]['Product_Rating']).toarray()
      num_array2 = data[data['Title']==name].select_dtypes(include=np.number).to_numpy()

      # Calculating similarities for text as well as numeric features
      text_sim = cosine_similarity(text_array1, text_array2)[0][0]
      num_sim = cosine_similarity(num_array1, num_array2)[0][0]
      sim.append(text_sim + num_sim)

  return sim

In [33]:
def recommend_games(games_name, data=games):
  # Base case
  if games[games['Title'] == games_name].shape[0] == 0:
    print('This game is either not so popular or you\
    have entered invalid_name.\n Some games you may like:\n')

    for game in data.sample(n=5)['Title'].values:
      print(game)
    return

  data['similarity_factor'] = get_similarities(games_name, data)

  data.sort_values(by=['similarity_factor', 'Product_Rating'],
                   ascending = [False, False],
                   inplace=True)

  # First game will be the input game itself as the similarity will be highest.
  display(data[['Title', 'Release_Date', 'Developer', 'Genres', 'Product_Rating', 'User_Score', 'Metric_Rating', 'similarity_factor']][2:10])

In [34]:
recommend_games('The Elder Scrolls V: Skyrim')

,Title,Release_Date,Developer,Genres,Product_Rating,User_Score,Metric_Rating,similarity_factor
1043,Super Mario Galaxy,11/12/2007,Nintendo,3D Platformer,E,91,97,0.999984
1069,Brothers in Arms: Road to Hill 30,3/1/2005,Gearbox Software,Tactical FPS,M,79,87,0.999972
1054,Neon Shadow,11/14/2013,Tasty Poison Games,FPS,RP,80,80,0.999970
1074,Wipeout HD Fury,7/23/2009,Studio Liverpool,Future Racing,E+10,80,89,0.999969
1067,Onimusha: Warlords,1/15/2019,NeoBards Entertainment,Action Adventure,M,72,86,0.999968
1055,The King of Fighters XI,11/13/2007,SNK Playmore,2D Fighting,T,85,75,0.999958
1037,Viewtiful Joe 2,11/18/2004,Clover Studio,2D Beat-'Em-Up,T,75,86,0.999958
1063,Amnesia Collection,11/22/2016,Frictional Games,Compilation,M,75,78,0.999955
